In [1]:
#https://www.machinelearningplus.com/time-series/vector-autoregression-examples-python/
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [2]:
df = pd.read_excel('data - aggregated daily columns - NORMALIZED - redo fb data - 07DEC2022.xlsx',index_col='date')

In [3]:
len(df)

549

In [4]:
df.tail()

,SN,new_cases,cumulative_cases,new_deaths,cumulative_deaths,announcement_count,fb_count,fb_fear_count,fb_fear_percentage,fb_fear_intensity,...,normalized_moh_religion,normalized_moh_travelborder,normalized_moh_vaccination,normalized_moh_heightenedalert,normalized_moh_travel,normalized_moh_reopening,normalized_moh_safemanagementmeasures,normalized_moh_education,normalized_moh_ministertranscript,normalized_imh_visits
date,,,,,,,,,,,,,,,,,,,,,
2021-12-27,545,209,277764,1,822,0,8484,1916,22.583687,0.532865,...,0.0,-0.047668,7.162729e-18,0.0,0.0,0.0,0.0,0.0,0.028912,2.534942
2021-12-28,546,280,278044,3,825,0,4431,1166,26.314602,0.527206,...,0.0,-0.047668,7.162729e-18,0.0,0.0,0.0,0.0,0.0,0.028912,-1.464011
2021-12-29,547,365,278409,0,825,0,4668,971,20.801200,0.524485,...,0.0,-0.047668,7.162729e-18,0.0,0.0,0.0,0.0,0.0,0.028912,0.045028
2021-12-30,548,341,278750,1,826,0,3942,1037,26.306443,0.526154,...,0.0,-0.047668,7.162729e-18,0.0,0.0,0.0,0.0,0.0,0.028912,0.422288
2021-12-31,549,311,279061,1,827,1,1678,439,26.162098,0.527907,...,0.0,2.907736,7.162729e-18,0.0,0.0,0.0,0.0,0.0,0.028912,-1.086751


In [5]:
pd.set_option('display.max_seq_items', None)
df.columns

Index(['SN', 'new_cases', 'cumulative_cases', 'new_deaths',
       'cumulative_deaths', 'announcement_count', 'fb_count', 'fb_fear_count',
       'fb_fear_percentage', 'fb_fear_intensity', 'fb_anger_count',
       'fb_anger_percentage', 'fb_anger_intensity', 'fb_sadness_count',
       'fb_sadness_percentage', 'fb_sadness_intensity', 'fb_joy_count',
       'fb_joy_percentage', 'fb_joy_intensity', 'fb_noemotion_count',
       'fb_noemotion_percentage', 'fb_noemotion_intensity', 'tweet_count',
       'tweet_fear_count', 'tweet_fear_percentage', 'tweet_fear_intensity',
       'tweet_anger_count', 'tweet_anger_percentage', 'tweet_anger_intensity',
       'tweet_sadness_count', 'tweet_sadness_percentage',
       'tweet_sadness_intensity', 'tweet_joy_count', 'tweet_joy_percentage',
       'tweet_joy_intensity', 'tweet_noemotion_count',
       'tweet_noemotion_percentage', 'tweet_noemotion_intensity',
       'mindline_total_users', 'mindline_new_users',
       'mindline_well_total_users', 'min

In [6]:
df_rel=df[[ 'normalized_new_cases', 'normalized_cumulative_cases',
       'normalized_new_deaths', 'normalized_cumulative_deaths',
       'normalized_announcement_count', 'normalized_fb_count',
       'normalized_fb_fear_count', 'normalized_fb_fear_percentage',
       'normalized_fb_fear_intensity', 'normalized_fb_anger_count',
       'normalized_fb_anger_percentage', 'normalized_fb_anger_intensity',
       'normalized_fb_sadness_count', 'normalized_fb_sadness_percentage',
       'normalized_fb_sadness_intensity', 'normalized_fb_joy_count',
       'normalized_fb_joy_percentage', 'normalized_fb_joy_intensity',
       'normalized_fb_noemotion_count',
       'normalized_fb_noemotion_percentage', 'normalized_fb_noemotion_intensity',            
       'normalized_tweet_count', 'normalized_tweet_fear_count',
       'normalized_tweet_fear_percentage', 'normalized_tweet_fear_intensity',
       'normalized_tweet_anger_count', 'normalized_tweet_anger_percentage',
       'normalized_tweet_anger_intensity', 'normalized_tweet_sadness_count',
       'normalized_tweet_sadness_percentage',
       'normalized_tweet_sadness_intensity', 'normalized_tweet_joy_count',
       'normalized_tweet_joy_percentage', 'normalized_tweet_joy_intensity',
       'normalized_tweet_noemotion_count',
       'normalized_tweet_noemotion_percentage', 'normalized_tweet_noemotion_intensity',
       'normalized_mindline_total_users', 'normalized_mindline_new_users',
       'normalized_mindline_well_total_users',
       'normalized_mindline_well_new_users',
       'normalized_mindline_mild_total_users',
       'normalized_mindline_mild_new_users',
       'normalized_mindline_moderate_total_users',
       'normalized_mindline_moderate_new_users',
       'normalized_mindline_crisis_total_users',
       'normalized_mindline_crisis_new_users',
       'normalized_moh_biz_advisory', 'normalized_moh_caseupdate',
       #'normalized_moh_closure',
       'normalized_moh_comm_surv_test', 'normalized_moh_discip_measures',
       'normalized_moh_govt_support', 
       'normalized_moh_migrantworker', 'normalized_moh_nonewcase',
       'normalized_moh_precautionary', 'normalized_moh_publicawareness',
       'normalized_moh_religion', #'normalized_moh_shn',
       #'normalized_moh_tracetogether', 'normalized_moh_travelborder',
       'normalized_moh_vaccination', 'normalized_moh_heightenedalert',
       'normalized_moh_travel', 'normalized_moh_reopening',
       'normalized_moh_safemanagementmeasures', 'normalized_moh_education',
       'normalized_moh_ministertranscript','normalized_imh_visits'  
          ]]

In [7]:
nobs = 10
df_train, df_test = df_rel[0:-nobs], df_rel[-nobs:]

# Check size
print(df_train.shape) 
print(df_test.shape) 

(539, 65)
(10, 65)


In [8]:
###adf function
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")    

In [9]:
### moh_circuitbreaker and moh_earlyphase are nonstationary (might need to drop)

In [10]:
#df_train['normalized_moh_circuitbreaker'] = df_train['normalized_moh_circuitbreaker'].diff()
#df_train['normalized_moh_circuitbreaker'].fillna(0, inplace=True)

In [11]:
#df_train['normalized_moh_earlyphase'] = df_train['normalized_moh_earlyphase'].diff()
#df_train['normalized_moh_earlyphase'].fillna(0, inplace=True)

In [12]:
# ADF Test on each column
for name, column in df_train.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "normalized_new_cases" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -4.5851
 No. Lags Chosen       = 19
 Critical value 1%     = -3.443
 Critical value 5%     = -2.867
 Critical value 10%    = -2.57
 => P-Value = 0.0001. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "normalized_cumulative_cases" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -3.199
 No. Lags Chosen       = 15
 Critical value 1%     = -3.443
 Critical value 5%     = -2.867
 Critical value 10%    = -2.57
 => P-Value = 0.02. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "normalized_new_deaths" 
    -----------------------------------------------
 Null Hypothesis: Data ha

In [13]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=5
test = 'ssr_chi2test'
def grangers_causation_matrix_new(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    Y = [];
    X = [];
    Vals  = [];
    df_rel = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df_rel.columns:
        for r in df_rel.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            
            
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            idx = p_values.index(min_p_value);
            df_rel.loc[r, c] = min_p_value; #
            if min_p_value < 0.05:
                Y.append(r);
                X.append(c);
                Vals.append(test_result[idx+1][0][test]);
    df_rel.columns = [var + '_x' for var in variables]
    df_rel.index = [var + '_y' for var in variables]
    return df_rel, Y, X, Vals
df_rel, Y, X, Vals = grangers_causation_matrix_new(df_rel, df_rel.columns, test='ssr_chi2test', verbose=False)

out = pd.DataFrame(data={"Y":Y, "X":X, "Val": Vals});#'Chi2':chi2_val, "p":p_val, "df_val":df_val})
out.to_csv('outputfile_latest_fbdata2.csv')
df_rel.to_csv('dfmatrix_latest_fbdata2.csv')


